<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Собственные уровни энергии через диагонализацию матрицы
## Examples - Quantum Mechanics
<section class="post-meta">
By Henning G. Hugdal, Håkon W. Ånes and Jon Andreas Støvneng
</section>
Last edited: January 19th 2019
___

В нескольких предыдущих примерах мы нашли собственные уровни энергии системы с помощью метода стрельбы. Однако в этом примере мы найдем собственные энергии системы, переформулировав задачу в линейную систему уравнений, и найдем энергии, через собственные значения результирующей матрицы, используя функции в `numpy.linalg` [[1]](#rsc). Мы начинаем с дискретизации уравнения Шредингера, чтобы получить линейную систему уравнений.

Уравнение Шредингера в одном измерении гласит

\begin{equation}
-\frac{\hbar^2}{2m}\frac{d^2}{d x^2} \psi(x) + V(x)\psi(x) = E\psi(x).
\end{equation}

Прежде всего, мы дискретизируем ось $x$, позволяя $x \rightarrow x_i = x_{min} + i\cdot \Delta x$, где $\Delta x = (x_{max}-x_{min})/(N-1)$, $ i \in [0, N-1] $ и $N$ - количество точек сетки. Мы также используем обозначения для значений функций с дискретными аргументами, $\psi(x_i) = \psi_i$ и $V(x_i) = V_i$.
Использование центральных различий второго порядка для второго производного члена,
$$ \frac{d^2}{d x^2}\psi(x_i) \rightarrow \frac{\psi_{i+1}-2\psi_i+\psi_{i-1}}{\Delta x^2},$$
наконец мы получаем дискретизированное уравнение Шредингера,

\begin{equation}
-\frac{\hbar^2}{2m}\frac{\psi_{i+1}-2\psi_i+\psi_{i-1}}{\Delta x^2} + V_i \psi_i = E \psi_i.
\end{equation}

В следующем мы установим $\hbar=m=1$ для простоты. Это означает, что матрица, описывающая нашу систему, будет иметь $1/\Delta x^2 + V_i$ вдоль главной диагонали и $-1/\Delta_x^2$ вдоль первых диагоналей ниже и выше главной диагонали. Обозначая матрицу, содержащую эти элементы $\bar{H}$, и вектор $\psi_i$-тых как $\vec{\psi}$, мы получаем следующую линейную систему уравнений:
$$\bar{H}\vec{\psi} = E\vec{\psi}.$$

### Колебания гармонического осциллятора

Сначала мы решим эту систему уравнений, используя потенциал гармонического осциллятора,
$$ V(x) = \frac{1}{2}x^2,$$
где $m$ и $\omega$ равны $1$. Обратите внимание, что для решения этой проблемы мы ограничим значения на оси $x$ конечным интервалом, даже если волновая функция в принципе может быть ненулевой на всей оси $x$. Однако для собственных состояний с наименьшей энергией это не очень строгое приближение. Обратите также внимание, что, поскольку $\bar{H}$ является $N\times N$-матрицей, это означает, что мы найдем только $N$ наименьших собственных энергий.

Давайте импортируем необходимые библиотеки и определим матрицу $\bar{H}$.

In [ ]:
from __future__ import division
import numpy as np
import numpy.linalg as linalg
import matplotlib.pyplot as plt


# Number of grid points
N = 100

# Interval on the x-axis:
x_min = -5
x_max = 5

dx = (x_max-x_min)/(N-1)

# Grid points
x = np.arange(x_min, x_max+dx, dx)

# Potential
V = 0.5*x**2

# We define a N-by-N zero matrix:
H = np.zeros((N, N))

# Set the non-zero elements of the matrix:
for i in range(N):
    H[i][i] = (1/dx**2+V[i])
    if i > 0:
        H[i][i-1] = -1/(2*dx**2)
    if i < N-1:
        H[i][i+1] = -1/(2*dx**2)

Поскольку $\bar{H}$ симметричен, мы можем использовать функцию linalg `eigvalsh(H)`, которая возвращает собственные значения $\bar{H}$ [[1]](#rsc). Ниже вычисляются собственные значения, и записываются 10 первых собственных значений.

In [ ]:
E = linalg.eigvalsh(H)

# Print the 10 first eigenvalues:
for i in range(10):
    Estr = "E" + str(i) + ": "
    print(Estr, E[i])

Мы видим, что это очень хорошо соответствует точным энергиям гармонического генератора, $E_n = (\frac{1}{2} + n)$. Однако обратите внимание, что ошибка увеличивается с увеличением энергий, поскольку приближение, выполняемое при ограничении используемого интервала $x$, ухудшается для более высоких собственных состояний.

### Асимметричный потенциал

В блокноте "[Численное определение собственных энергий для асимметричного потенциале](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/asymmetric_potential.ipynb)" мы нашли собственные энергии для трех самых низких собственных состояний для асимметричного потенциала,

\begin{equation}
V(x) = ax^4 - b(x+c)^2 + d,
\end{equation}

с $a=1$, $b=1,5$, $c=0,2$ и $d=1,17$. В качестве контроля этих результатов мы можем использовать метод, использованный выше, для определения тех же собственных энергий. Здесь мы также определим собственные функции, чтобы сравнить их с теми, которые были получены в упомянутой выше записной книжке. Это означает использование `linalg.eigh()` вместо `linalg.eigvalsh()`.

In [ ]:
# Use the same values as in Example 6.6
N = 1000
x_min = -10
x_max = 10
dx = (x_max - x_min)/(N-1)
x = np.arange(x_min, x_max+dx, dx)

# Set the potential
a = 1
b = 1.5
c = 0.2
d = 1.17

V = a*x**4 - b*(x + c)**2 + d

# We define a N-by-N zero matrix:
H = np.zeros((N, N))

# Set the non-zero elements of the matrix:
for i in range(N):
    H[i][i] = (1/dx**2+V[i])
    if i > 0:
        H[i][i-1] = -1/(2*dx**2)
    if i < N-1:
        H[i][i+1] = -1/(2*dx**2)

# Determine the eigenenergies and the eigenvectors
E, psi = linalg.eigh(H)

# Print the 3 first eigenvalues:
for i in range(3):
    Estr = "E" + str(i) + ": "
    print(Estr, E[i])

Мы видим, что это очень хорошо соответствует значениям, приведенным в примере 6.6. Ниже мы построим собственные функции трех наименьших собственных энергий.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Set common figure parameters:
newparams = {'axes.labelsize': 14, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (8, 3),
             'figure.subplot.wspace': 0.4,
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 10, 'legend.frameon': False, 
             'legend.handlelength': 1.5, 'figure.dpi': 200}
plt.rcParams.update(newparams)

# Plot three lowest eigenstates
plt.figure()
plt.title("Ground state")
plt.plot(x, psi[:, 0])
plt.xlim([-3, 3])
plt.grid()
plt.figure()
plt.title("First excited state")
plt.plot(x, -psi[:, 1])
plt.xlim([-3, 3])
plt.grid()
plt.figure()
plt.title("Second excited state")
plt.plot(x, psi[:, 2])
plt.xlim([-3, 3])
plt.grid();

Мы также видим, что собственные функции хорошо соответствуют тем, которые были определены в предыдущем блокноте. 

Следует также отметить, что это гораздо более эффективный и простой способ решения вышеуказанной проблемы!

<a id="rsc"></a>
___
## References

<a>[1]</a> `numpy.linalg` [documentation](http://docs.scipy.org/doc/numpy/reference/routines.linalg.html)